# Ice Cream Sales Prediction

This notebook aims to predict ice cream sales based on historical weather data. The workflow includes data preprocessing, feature engineering, model training, and evaluation. The steps are as follows:

1. Import necessary libraries and install required packages.
2. Load and inspect historical weather and sales data.
3. Preprocess the data, including handling missing values and converting date formats.
4. Aggregate weather data to daily level and merge with sales data.
5. Train a multi-output regression model to predict sales.
6. Evaluate the model's performance using metrics like Mean Squared Error (MSE) and Root Mean Squared Error (RMSE).
7. Save the trained model for future use.

#### Dependicies: 

In [1]:
!pip install pandas
!pip install scikit-learn
!pip install joblib
!pip install numpy

In [54]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error

* Load the data from the csv file:

In [55]:
weather = pd.read_csv('data/temp-history.csv')
sales = pd.read_csv("data/Ajustes de Stock Rio.csv", sep=",", skipinitialspace=True)

In [56]:
weather.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1262304000,2010-01-01 00:00:00 +0000 UTC,-10800,Rosario,-32.958702,-60.693042,23.74,10000.0,14.99,23.68,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
1,1262307600,2010-01-01 01:00:00 +0000 UTC,-10800,Rosario,-32.958702,-60.693042,23.50,NaN,17.72,23.73,...,NaN,NaN,NaN,NaN,NaN,15,801,Clouds,few clouds,02n
2,1262311200,2010-01-01 02:00:00 +0000 UTC,-10800,Rosario,-32.958702,-60.693042,22.24,NaN,17.60,22.48,...,NaN,NaN,NaN,NaN,NaN,4,800,Clear,sky is clear,01n
3,1262314800,2010-01-01 03:00:00 +0000 UTC,-10800,Rosario,-32.958702,-60.693042,21.81,NaN,17.81,22.08,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
4,1262318400,2010-01-01 04:00:00 +0000 UTC,-10800,Rosario,-32.958702,-60.693042,21.51,NaN,17.51,21.75,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n


In [57]:
sales.head()

,Ajuste Fecha,Dia Semana,Americana,Americana Dietetico,Banana con Dulce,Banana sin tacc x 16 unidades,Banana sin tacc x 27 unidades,Bombón Rocher,Candy Americana,Candy Chocolate,...,Roll clasico (frutilla-dulce-choco),Super Sambayón,Super nute,Tiramisú Italiano,Torta Chocotorta,Torta Oreo,Tramontana,Vainilla,Unnamed: 81,Total Cantidad
0,08/07/2019 00:00:00,Lunes,20.61,38.54,22.36,20.70,15.74,17.69,47.42,25.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09/07/2019 00:00:00,Martes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/07/2019 00:00:00,Miércoles,NaN,-5.56,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/07/2019 00:00:00,Jueves,-4.84,NaN,NaN,NaN,NaN,NaN,-5.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/07/2019 00:00:00,Viernes,21.50,19.28,33.16,24.78,15.33,NaN,37.62,12.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Converting the `Dia Semana` to day_of_week in the sales df:

In [58]:
sales.rename(columns={'Dia Semana':'day_of_week'}, inplace=True)

sales['day_of_week'] = sales['day_of_week'].map({'Lunes':1, 'Martes':2, 'Miércoles':3, 'Jueves':4, 'Viernes':5, 'Sábado':6, 'Domingo':7})

In [59]:
sales.head()

,Ajuste Fecha,day_of_week,Americana,Americana Dietetico,Banana con Dulce,Banana sin tacc x 16 unidades,Banana sin tacc x 27 unidades,Bombón Rocher,Candy Americana,Candy Chocolate,...,Roll clasico (frutilla-dulce-choco),Super Sambayón,Super nute,Tiramisú Italiano,Torta Chocotorta,Torta Oreo,Tramontana,Vainilla,Unnamed: 81,Total Cantidad
0,08/07/2019 00:00:00,1,20.61,38.54,22.36,20.70,15.74,17.69,47.42,25.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09/07/2019 00:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/07/2019 00:00:00,3,NaN,-5.56,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/07/2019 00:00:00,4,-4.84,NaN,NaN,NaN,NaN,NaN,-5.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/07/2019 00:00:00,5,21.50,19.28,33.16,24.78,15.33,NaN,37.62,12.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Removing the trailing " UTC" from the datetime string and parse it


In [60]:
weather['dt_iso'] = pd.to_datetime(
    weather['dt_iso'].str.replace(' UTC', '', regex=False),
    format='%Y-%m-%d %H:%M:%S %z'
)

In [61]:
weather['dt_iso'].head()

0   2010-01-01 00:00:00+00:00
1   2010-01-01 01:00:00+00:00
2   2010-01-01 02:00:00+00:00
3   2010-01-01 03:00:00+00:00
4   2010-01-01 04:00:00+00:00
Name: dt_iso, dtype: datetime64[ns, UTC]

In [62]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136396 entries, 0 to 136395
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   dt                   136396 non-null  int64              
 1   dt_iso               136396 non-null  datetime64[ns, UTC]
 2   timezone             136396 non-null  int64              
 3   city_name            136396 non-null  object             
 4   lat                  136396 non-null  float64            
 5   lon                  136396 non-null  float64            
 6   temp                 136396 non-null  float64            
 7   visibility           113825 non-null  float64            
 8   dew_point            136396 non-null  float64            
 9   feels_like           136396 non-null  float64            
 10  temp_min             136396 non-null  float64            
 11  temp_max             136396 non-null  float64            
 12  pr

In [63]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2033 entries, 0 to 2032
Data columns (total 83 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Ajuste Fecha                                     2033 non-null   object 
 1   day_of_week                                      2033 non-null   int64  
 2   Americana                                        1803 non-null   float64
 3   Americana Dietetico                              1090 non-null   float64
 4   Banana con Dulce                                 1853 non-null   float64
 5   Banana sin tacc x 16 unidades                    614 non-null    float64
 6   Banana sin tacc x 27 unidades                    516 non-null    float64
 7   Bombón Rocher                                    1676 non-null   float64
 8   Candy Americana                                  1778 non-null   float64
 9   Candy Chocolate               

* Checking and handling missing values: 

In [64]:
sales.isnull().sum()

Ajuste Fecha              0
day_of_week               0
Americana               230
Americana Dietetico     943
Banana con Dulce        180
                       ... 
Torta Oreo             1185
Tramontana             1118
Vainilla               1235
Unnamed: 81            2031
Total Cantidad          934
Length: 83, dtype: int64

In [65]:
weather.isnull().sum()

dt                          0
dt_iso                      0
timezone                    0
city_name                   0
lat                         0
lon                         0
temp                        0
visibility              22571
dew_point                   0
feels_like                  0
temp_min                    0
temp_max                    0
pressure                    0
sea_level              136396
grnd_level             136396
humidity                    0
wind_speed                  0
wind_deg                    0
wind_gust              117989
rain_1h                122199
rain_3h                136387
snow_1h                136396
snow_3h                136396
clouds_all                  0
weather_id                  0
weather_main                0
weather_description         0
weather_icon                0
dtype: int64

In [67]:
sales.head()

,Ajuste Fecha,day_of_week,Americana,Americana Dietetico,Banana con Dulce,Banana sin tacc x 16 unidades,Banana sin tacc x 27 unidades,Bombón Rocher,Candy Americana,Candy Chocolate,...,Roll clasico (frutilla-dulce-choco),Super Sambayón,Super nute,Tiramisú Italiano,Torta Chocotorta,Torta Oreo,Tramontana,Vainilla,Unnamed: 81,Total Cantidad
0,08/07/2019 00:00:00,1,20.61,38.54,22.36,20.70,15.74,17.69,47.42,25.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09/07/2019 00:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/07/2019 00:00:00,3,NaN,-5.56,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/07/2019 00:00:00,4,-4.84,NaN,NaN,NaN,NaN,NaN,-5.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/07/2019 00:00:00,5,21.50,19.28,33.16,24.78,15.33,NaN,37.62,12.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
weather.fillna(0, inplace=True)

In [69]:
sales.fillna(0, inplace=True)

In [70]:
weather.isnull().sum()

dt                     0
dt_iso                 0
timezone               0
city_name              0
lat                    0
lon                    0
temp                   0
visibility             0
dew_point              0
feels_like             0
temp_min               0
temp_max               0
pressure               0
sea_level              0
grnd_level             0
humidity               0
wind_speed             0
wind_deg               0
wind_gust              0
rain_1h                0
rain_3h                0
snow_1h                0
snow_3h                0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

In [71]:
sales.isnull().sum()

Ajuste Fecha           0
day_of_week            0
Americana              0
Americana Dietetico    0
Banana con Dulce       0
                      ..
Torta Oreo             0
Tramontana             0
Vainilla               0
Unnamed: 81            0
Total Cantidad         0
Length: 83, dtype: int64

* Checking the data types of the columns:

In [72]:
weather['dt_iso'] = pd.to_datetime(weather['dt_iso'], errors='coerce')

weather['date'] = weather['dt_iso'].dt.date

* Aggregating the weather data to daily level:

In [74]:
daily_weather = weather.groupby('date').agg({
    'temp': 'mean',
    'feels_like': 'mean',
    'temp_min': 'mean',
    'temp_max': 'mean',
    'humidity': 'mean',
    'dew_point': 'mean',
    'wind_speed': 'mean',
    'wind_deg': 'mean',
    'clouds_all': 'mean',
    'visibility': 'mean',  
    'rain_1h': 'sum',     
    'rain_3h': 'sum',
    'snow_1h': 'sum',
    'snow_3h': 'sum'
}).reset_index()

In [75]:
daily_weather.head()

,date,temp,feels_like,temp_min,temp_max,humidity,dew_point,wind_speed,wind_deg,clouds_all,visibility,rain_1h,rain_3h,snow_1h,snow_3h
0,2010-01-01,22.802500,23.119583,22.435000,23.179583,77.500000,18.492500,2.862083,169.458333,35.041667,4916.666667,0.16,0.0,0.0,0.0
1,2010-01-02,24.720417,25.720833,24.187083,25.185833,75.583333,19.517500,0.974167,48.041667,0.666667,5583.333333,0.00,0.0,0.0,0.0
2,2010-01-03,25.569167,26.761250,25.047917,26.164583,74.500000,20.421667,2.819167,69.375000,24.750000,8333.333333,0.12,0.0,0.0,0.0
3,2010-01-04,26.676000,29.145600,26.210000,27.185600,76.200000,21.949200,2.868000,118.040000,61.960000,5480.000000,5.76,0.0,0.0,0.0
4,2010-01-05,28.104167,31.863750,27.541250,28.615000,83.333333,24.820417,1.377917,90.958333,22.625000,7291.666667,2.56,0.0,0.0,0.0


In [77]:
sales['Ajuste Fecha'] = pd.to_datetime(sales['Ajuste Fecha'], dayfirst=True)
sales.rename(columns={'Ajuste Fecha': 'date'}, inplace=True)
sales['date'] = sales['date'].dt.date

* Merging the weather data with sales data:

In [78]:
df = pd.merge(sales, daily_weather, on='date', how='inner')

In [79]:
df.head()

,date,day_of_week,Americana,Americana Dietetico,Banana con Dulce,Banana sin tacc x 16 unidades,Banana sin tacc x 27 unidades,Bombón Rocher,Candy Americana,Candy Chocolate,...,humidity,dew_point,wind_speed,wind_deg,clouds_all,visibility,rain_1h,rain_3h,snow_1h,snow_3h
0,2019-07-08,1,20.61,38.54,22.36,20.70,15.74,17.69,47.42,25.69,...,72.208333,3.419167,2.737500,82.500000,0.000000,10000.000000,0.00,0.0,0.0,0.0
1,2019-07-09,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,73.166667,7.383333,2.114583,73.625000,15.833333,8000.000000,0.00,0.0,0.0,0.0
2,2019-07-10,3,0.00,-5.56,0.00,0.00,0.00,0.00,0.00,0.00,...,74.416667,9.566667,2.000000,101.666667,0.000000,9833.333333,0.00,0.0,0.0,0.0
3,2019-07-12,4,-4.84,0.00,0.00,0.00,0.00,0.00,-5.41,0.00,...,79.280000,10.394800,4.568000,172.400000,36.000000,8319.960000,3.78,0.0,0.0,0.0
4,2019-07-13,5,21.50,19.28,33.16,24.78,15.33,0.00,37.62,12.44,...,84.541667,6.105833,3.029167,147.083333,0.000000,8083.333333,0.00,0.0,0.0,0.0


* Feacture cols: 

In [80]:
feature_columns = [
    "day_of_week","temp", "feels_like", "temp_min", "temp_max",
    "humidity", "dew_point",
    "wind_speed", "wind_deg",
    "clouds_all", "visibility",
    "rain_1h", "rain_3h", "snow_1h", "snow_3h"
]

* Splitting the data into training and testing sets:

In [86]:
X = df[feature_columns]
y = df
numeric_cols = y.select_dtypes(include=['number']).columns
y = y[numeric_cols]

In [88]:
numeric_cols = y.select_dtypes(include=['number']).columns
y.abs()

,day_of_week,Americana,Americana Dietetico,Banana con Dulce,Banana sin tacc x 16 unidades,Banana sin tacc x 27 unidades,Bombón Rocher,Candy Americana,Candy Chocolate,Cheesecake de Frambuesa,...,humidity,dew_point,wind_speed,wind_deg,clouds_all,visibility,rain_1h,rain_3h,snow_1h,snow_3h
0,1,20.61,38.54,22.36,20.70,15.74,17.69,47.42,25.69,0.00,...,72.208333,3.419167,2.737500,82.500000,0.000000,10000.000000,0.00,0.00,0.0,0.0
1,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,73.166667,7.383333,2.114583,73.625000,15.833333,8000.000000,0.00,0.00,0.0,0.0
2,3,0.00,5.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,74.416667,9.566667,2.000000,101.666667,0.000000,9833.333333,0.00,0.00,0.0,0.0
3,4,4.84,0.00,0.00,0.00,0.00,0.00,5.41,0.00,0.00,...,79.280000,10.394800,4.568000,172.400000,36.000000,8319.960000,3.78,0.00,0.0,0.0
4,5,21.50,19.28,33.16,24.78,15.33,0.00,37.62,12.44,0.00,...,84.541667,6.105833,3.029167,147.083333,0.000000,8083.333333,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,3,10.17,10.22,15.06,0.00,0.00,17.60,9.68,19.47,0.00,...,50.125000,16.731667,2.885417,141.958333,4.416667,4583.333333,0.00,0.00,0.0,0.0
2026,4,15.59,0.00,0.00,0.00,0.00,12.26,4.84,0.00,11.58,...,63.041667,19.212917,3.186667,153.625000,29.166667,7500.000000,1.90,2.64,0.0,0.0
2027,5,5.39,0.00,19.75,0.00,0.00,18.35,0.00,0.00,18.24,...,53.416667,14.532500,2.743750,148.708333,18.458333,4333.333333,0.00,0.00,0.0,0.0
2028,6,18.01,11.69,18.88,0.00,0.00,23.35,33.94,14.59,5.67,...,56.125000,17.700417,3.646667,94.458333,0.833333,7916.666667,0.00,0.00,0.0,0.0


In [89]:
model = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=200, 
    min_samples_leaf=1,
    random_state=42,
    bootstrap=True,
))
model.fit(X, y)

MultiOutputRegressor(estimator=RandomForestRegressor(n_estimators=200,
                                                     random_state=42))

In [90]:
def safe_predict(self, X):
    predictions = self.predict(X)
    return np.maximum(predictions, 0) 

In [91]:
MultiOutputRegressor.safe_predict = safe_predict

In [94]:
joblib.dump(model, 'model/ice_cream_sales_model.pkl')

['/Users/aaqibnazir/Documents/work/ScoopCast/model/ice_cream_sales_model.pkl']

In [92]:
accuracy = model.score(X, y)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9001314261104922


In [30]:
y_pred = model.safe_predict(X)
mse = mean_squared_error(y, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 25.570142736403287


In [31]:
rmse = mse**0.5
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 5.056692865540015
